# Making Your Model Learn Addition!
## Task 1: Introduction

Given the string "54+7", the model should return a prediction: "61".

In [1]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Dense, Dropout, SimpleRNN, RepeatVector
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback

from termcolor import colored

## Task 2: Generate Data
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [4]:
all_chars = '0123456789+'

In [6]:
num_features = len(all_chars)
print("number of features:", num_features)
char_to_index = dict((c, i) for i, c in enumerate(all_chars))
index_to_char = dict((i, c) for i, c in enumerate(all_chars))

number of features: 11


In [8]:
def generate_data():
    first = np.random.randint(0, 100)
    second = np.random.randint(0, 100)
    example = str(first) + "+" + str(second)
    label = str(first + second)
    return example, label

generate_data()

('38+58', '96')

## Task 3: Create the Model
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___
Consider these two reviews:

Review 1: This movie is not terrible at all.

Review 2: This movie is pretty decent.

In [11]:
hidden_units = 128
max_time_steps = 5
model = Sequential([
    SimpleRNN(hidden_units, input_shape=(None, num_features)),
    RepeatVector(max_time_steps),
    SimpleRNN(hidden_units, return_sequences=True),
    TimeDistributed(Dense(num_features, activation='softmax'))
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_3 (SimpleRNN)     (None, 128)               17920     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 5, 128)            0         
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 5, 128)            32896     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 11)             1419      
Total params: 52,235
Trainable params: 52,235
Non-trainable params: 0
_________________________________________________________________


## Task 4: Vectorize and De-Vectorize Data
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [13]:
def vectorize_example(example, label):
    x = np.zeros((max_time_steps, num_features))
    y = np.zeros((max_time_steps, num_features))
    
    diff_x = max_time_steps - len(example)
    diff_y = max_time_steps - len(label)
    
    for i, c in enumerate(example):
        x[i+diff_x, char_to_index[c]] = 1
    for i in range(diff_x):
        x[i, char_to_index['0']] = 1
    for i, c in enumerate(label):
        y[i+diff_y, char_to_index[c]] = 1
    for i in range(diff_y):
        y[i, char_to_index['0']] = 1
    return x, y

e, l = generate_data()
print(e, l)
x, y = vectorize_example(e, l)
print(x.shape, y.shape)

17+39 56
(5, 11) (5, 11)


In [15]:
x, y

(array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]]),
 array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]]))

In [16]:
def devectorize_example(example):
    result = [index_to_char[np.argmax(vec)] for vec in example]
    return ''.join(result)

devectorize_example(x)

'17+39'

In [17]:
devectorize_example(y)

'00056'

## Task 5: Create Dataset
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [19]:
def create_dataset(num_examples=2000):
    x = np.zeros((num_examples, max_time_steps, num_features))
    y = np.zeros((num_examples, max_time_steps, num_features))
    for i in range(num_examples):
        e, l = generate_data()
        e_v, l_v = vectorize_example(e, l)
        x[i] = e_v
        y[i] = l_v
    return x, y
x, y = create_dataset()
print(x.shape, y.shape)

(2000, 5, 11) (2000, 5, 11)


In [20]:
devectorize_example(x[0])

'52+93'

In [21]:
devectorize_example(y[0])

'00145'

## Task 6: Training the Model
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [22]:
l_cb = LambdaCallback(
    on_epoch_end=lambda e, l: print('{:.2f}'.format(l['val_acc']), end=' _ ')
)
es_cb = EarlyStopping(monitor='val_loss', patience=10)
model.fit(x, y, epochs=500, batch_size=256, validation_split=0.2, verbose=False, callbacks=[es_cb, l_cb])

0.55 _ 0.59 _ 0.61 _ 0.62 _ 0.62 _ 0.62 _ 0.64 _ 0.64 _ 0.64 _ 0.65 _ 0.65 _ 0.66 _ 0.66 _ 0.67 _ 0.67 _ 0.68 _ 0.68 _ 0.68 _ 0.69 _ 0.70 _ 0.71 _ 0.71 _ 0.71 _ 0.73 _ 0.73 _ 0.73 _ 0.73 _ 0.74 _ 0.75 _ 0.74 _ 0.75 _ 0.75 _ 0.75 _ 0.75 _ 0.75 _ 0.76 _ 0.76 _ 0.75 _ 0.76 _ 0.76 _ 0.76 _ 0.77 _ 0.77 _ 0.77 _ 0.77 _ 0.78 _ 0.79 _ 0.79 _ 0.79 _ 0.80 _ 0.80 _ 0.81 _ 0.81 _ 0.82 _ 0.83 _ 0.83 _ 0.83 _ 0.84 _ 0.85 _ 0.85 _ 0.85 _ 0.86 _ 0.86 _ 0.86 _ 0.87 _ 0.87 _ 0.87 _ 0.89 _ 0.88 _ 0.89 _ 0.88 _ 0.89 _ 0.90 _ 0.89 _ 0.90 _ 0.90 _ 0.90 _ 0.91 _ 0.92 _ 0.91 _ 0.91 _ 0.91 _ 0.92 _ 0.92 _ 0.92 _ 0.92 _ 0.93 _ 0.92 _ 0.93 _ 0.92 _ 0.93 _ 0.92 _ 0.92 _ 0.93 _ 0.93 _ 0.93 _ 0.92 _ 0.93 _ 0.93 _ 0.93 _ 0.94 _ 0.94 _ 0.94 _ 0.94 _ 0.94 _ 0.94 _ 0.95 _ 0.94 _ 0.95 _ 0.94 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.94 _ 0.95 _ 0.94 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.94 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.96 _ 0.95 _ 0.95 _ 0.96 _ 0.95 _ 0.95 _ 0.95 _

In [26]:
x_test, y_test = create_dataset(10)
preds = model.predict(x_test)

for i, pred in enumerate(preds):
    y = devectorize_example(y_test[i])
    y_hat = devectorize_example(pred)
    col = 'green'
    if y != y_hat:
        col = 'red'
    out = 'Input: ' + devectorize_example(x_test[i]) + ' Out: ' + y + ' Pred: ' + y_hat
    print(colored(out, col))

Input: 12+37 Out: 00049 Pred: 00049
Input: 53+91 Out: 00144 Pred: 00144
Input: 01+92 Out: 00093 Pred: 00093
Input: 13+46 Out: 00059 Pred: 00069
Input: 76+62 Out: 00138 Pred: 00138
Input: 97+93 Out: 00190 Pred: 00180
Input: 024+2 Out: 00026 Pred: 00025
Input: 86+10 Out: 00096 Pred: 00096
Input: 99+12 Out: 00111 Pred: 00111
Input: 64+64 Out: 00128 Pred: 00128
